# Other network shapes
We saw networks with a single input and output. But sometimes you would want to have more then one input, or output. That can be done with the Functional API:

In [9]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path

import numpy as np
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import Model
import tensorflow as tf

sys.path.insert(0, "..") 
from src.data import make_dataset


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
datafile = Path("..") / "data/processed/data.npy"
train, valid, test = make_dataset.load(datafile)
X_train, y_train = train

In [11]:
input1 = Input(shape=X_train.shape[1:])
input2 = Input(shape=X_train.shape[1:])
x1 = Dense(30, activation='relu')(input1)
x2 = Dense(30, activation='relu')(input2)
x = tf.concat([x1, x2], 1)
output = Dense(1)(x)

model = Model(inputs=[input1, input2], outputs=[output])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 30)           270         input_7[0][0]                    
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 30)           270         input_8[0][0]                    
____________________________________________________________________________________________

Here, we created two inputs, each going into their own, separate `Dense(30)` layer. Each of those outputs a vector of length 30. Note how we have to take care not to overwrite them. So we use `x1` and `x2`. The outputs are concatenated into a vector of lengthe 60, and fed into a `Dense(1)` layer. Let's say we want to output the x layer in an earlier stage, that can be done like this:

In [12]:
input1 = Input(shape=X_train.shape[1:])
input2 = Input(shape=X_train.shape[1:])
x1 = Dense(30, activation='relu')(input1)
x2 = Dense(30, activation='relu')(input2)
x = tf.concat([x1, x2], 1)
early_output = Dense(1)(x)
x = Dense(30, activation='relu')(x)
x = Dense(30, activation='relu')(x)
output = Dense(1)(x)

model = Model(inputs=[input1, input2], outputs=[output, early_output])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense_8 (Dense)                 (None, 30)           270         input_9[0][0]                    
__________________________________________________________________________________________________
dense_9 (Dense)                 (None, 30)           270         input_10[0][0]                   
____________________________________________________________________________________________

Note, that Tensorflow will use the same loss function for both outputs.  If you dont want that, you will need to specify multiple losses, and if you don't want the losses to be added, you should add weights too.

Also note, how we can reuse the variable `x` after we no longer have two datastreams we want to keep separate.

In [13]:
model.compile(loss=['mape', 'mse'], loss_weights=[0.9, 0.1])

However, these are already fairly complex models. Usually, you don't need to make things this complex. I just want you to know that you can do it like this, if you ever have the need to do so.

In this example, the `output` has a `mape` loss, the `early_output` an `mse` loss. The first one is weighted 0.9, the second one 0.1.